In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
#import and store the dataset
dataf = pd.read_csv('creditcard.csv')

In [3]:
#print(dataf)

In [4]:
# step 1. use shuffle/ randomize data

In [5]:
# step 2. use One-hot encoding 

In [6]:
# step 3. use Normalize

In [7]:
# step 4. use data spletting up X/y Values

In [8]:
# step 5. use conver data_frame to numpy arrays (float32)

In [9]:
# step 6. use spletting the final data into X/y train/test

In [10]:
shuffled_data = dataf.sample(frac=1)

In [11]:
#print (shuffled_data)

In [12]:
one_hot_data = pd.get_dummies(shuffled_data, columns=['Class'])

In [13]:
#print(one_hot_data)

In [14]:
normalized_data = (one_hot_data - one_hot_data.min()) / (one_hot_data.max() - one_hot_data.min())

In [15]:
#print(normalized_data)

In [16]:
df_X = normalized_data.drop(['Class_0', 'Class_1'], axis=1)

In [17]:
df_y = normalized_data[['Class_0', 'Class_1']]

In [18]:
# step 4. Spletting up X/y Values
ar_X, ar_y = np.asarray(df_X.values, dtype ='float32'), np.asarray(df_y.values, dtype='float32')

In [19]:
#Allocate first 80% of data_frames into training data and remining 20% to testing data
train_size = int(0.8 * len(ar_X))

In [20]:
(raw_X_train, raw_y_train) = (ar_X[:train_size], ar_y[:train_size])

In [21]:
(raw_X_test, raw_y_test) = (ar_X[train_size:], ar_y[train_size:])

In [23]:
count_legit, count_fraud = np.unique(dataf['Class'], return_counts=True) [1]

In [24]:
fraud_ratio = (count_fraud / (count_legit + count_fraud))

In [25]:
print('Percent of fraudulent transactions:', fraud_ratio)

Percent of fraudulent transactions: 0.001727485630620034


In [26]:
# Applies a logit weighting of 578 (1/0.0017) to fraudulent transactions to cause model to pay more attention to them
weighting = 1 / fraud_ratio
raw_y_train[:, 1] = raw_y_train[:, 1] * weighting

In [28]:
# importing older version of tensorflow as new version does not support GPU on macbook.
import tensorflow.compat.v1 as tf

In [29]:
# 30 cells for the input dimensions
input_dimensions = ar_X.shape[1]

In [30]:
# 2 cells for the output dimensions 
output_dimensions = ar_y.shape[1]

In [31]:
# 100 cells for the 1st layer
num_layer_1_cells = 100

In [32]:
# 150 cells for the second layer
num_layer_2_cells = 150

In [33]:
#to fix an issue with tf 2.0 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
#with tf.Session() as sess:
# We will use these as inputs to the model when it comes time to train it (assign values at run time)
X_train_node = tf.placeholder(tf.float32, [None, input_dimensions], name='X_train')
y_train_node = tf.placeholder(tf.float32, [None, output_dimensions], name='y_train')

Instructions for updating:
non-resource variables are not supported in the long term


In [34]:
# We will use these as inputs to the model once it comes time to test it
X_test_node = tf.constant(raw_X_test, name='X_test')
y_test_node = tf.constant(raw_y_test, name='y_test')

In [35]:
# First layer takes in input and passes output to 2nd layer
weight_1_node = tf.Variable(tf.zeros([input_dimensions, num_layer_1_cells]), name='weight_1')
biases_1_node = tf.Variable(tf.zeros([num_layer_1_cells]), name='biases_1')

In [36]:
# Second layer takes in input from 1st layer and passes output to 3rd layer
weight_2_node = tf.Variable(tf.zeros([num_layer_1_cells, num_layer_2_cells]), name='weight_2')
biases_2_node = tf.Variable(tf.zeros([num_layer_2_cells]), name='biases_2')

In [37]:
# Third layer takes in input from 2nd layer and outputs [1 0] or [0 1] depending on fraud vs legit
weight_3_node = tf.Variable(tf.zeros([num_layer_2_cells, output_dimensions]), name='weight_3')
biases_3_node = tf.Variable(tf.zeros([output_dimensions]), name='biases_3')

In [38]:
# Function to run an input tensor through the 3 layers and output a tensor that will give us a fraud/legit result
# Each layer uses a different function to fit lines through the data and predict whether a given input tensor will \
#   result in a fraudulent or legitimate transaction
def network(input_tensor):
    # Sigmoid fits modified data well
    layer1 = tf.nn.sigmoid(tf.matmul(input_tensor, weight_1_node) + biases_1_node)
    # Dropout prevents model from becoming lazy and over confident
    layer2 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(layer1, weight_2_node) + biases_2_node), 0.85)
    # Softmax works very well with one hot encoding which is how results are outputted
    layer3 = tf.nn.softmax(tf.matmul(layer2, weight_3_node) + biases_3_node)
    return layer3

In [39]:
# Used to predict what results will be given training or testing input data
# Remember, X_train_node is just a placeholder for now. We will enter values at run time
y_train_prediction = network(X_train_node)
y_test_prediction = network(X_test_node)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [40]:
# Cross entropy loss function measures differences between actual output and predicted output
cross_entropy = tf.losses.softmax_cross_entropy(y_train_node, y_train_prediction)

In [41]:
# Adam optimizer function will try to minimize loss (cross_entropy) but changing the 3 layers' variable values at a
#   learning rate of 0.005
optimizer = tf.train.AdamOptimizer(0.005).minimize(cross_entropy)

In [42]:
# Function to calculate the accuracy of the actual result vs the predicted result
def calculate_accuracy(actual, predicted):
    actual = np.argmax(actual, 1)
    predicted = np.argmax(predicted, 1)
    return (100 * np.sum(np.equal(predicted, actual)) / predicted.shape[0])

num_epochs = 100

In [43]:
import time

In [44]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(num_epochs):

        start_time = time.time()

        _, cross_entropy_score = session.run([optimizer, cross_entropy],
                                             feed_dict={X_train_node: raw_X_train, y_train_node: raw_y_train})

        if epoch % 10 == 0:
            timer = time.time() - start_time

            print('Epoch: {}'.format(epoch), 'Current loss: {0:.4f}'.format(cross_entropy_score),
                  'Elapsed time: {0:.2f} seconds'.format(timer))

            final_y_test = y_test_node.eval()
            final_y_test_prediction = y_test_prediction.eval()
            final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
            print("Current accuracy: {0:.2f}%".format(final_accuracy))

    final_y_test = y_test_node.eval()
    final_y_test_prediction = y_test_prediction.eval()
    final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
    print("Final accuracy: {0:.2f}%".format(final_accuracy))

final_fraud_y_test = final_y_test[final_y_test[:, 1] == 1]
final_fraud_y_test_prediction = final_y_test_prediction[final_y_test[:, 1] == 1]
final_fraud_accuracy = calculate_accuracy(final_fraud_y_test, final_fraud_y_test_prediction)
print('Final fraud specific accuracy: {0:.2f}%'.format(final_fraud_accuracy))

Epoch: 0 Current loss: 1.3999 Elapsed time: 0.79 seconds
Current accuracy: 0.16%
Epoch: 10 Current loss: 1.3982 Elapsed time: 0.66 seconds
Current accuracy: 9.08%
Epoch: 20 Current loss: 1.3797 Elapsed time: 0.67 seconds
Current accuracy: 0.88%
Epoch: 30 Current loss: 1.2868 Elapsed time: 0.66 seconds
Current accuracy: 66.78%
Epoch: 40 Current loss: 1.1216 Elapsed time: 0.66 seconds
Current accuracy: 96.48%
Epoch: 50 Current loss: 1.0005 Elapsed time: 0.69 seconds
Current accuracy: 98.54%
Epoch: 60 Current loss: 0.9232 Elapsed time: 0.67 seconds
Current accuracy: 99.21%
Epoch: 70 Current loss: 0.8899 Elapsed time: 0.66 seconds
Current accuracy: 99.50%
Epoch: 80 Current loss: 0.8717 Elapsed time: 0.66 seconds
Current accuracy: 99.60%
Epoch: 90 Current loss: 0.8589 Elapsed time: 0.66 seconds
Current accuracy: 99.71%
Final accuracy: 99.36%
Final fraud specific accuracy: 86.67%
